In [10]:
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
from contextlib import nullcontext

import pandas as pd

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-stock'
eval_interval = 10
log_interval = 1
eval_iters = 20
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'stock'
wandb_run_name = 'mini-gpt' # 'run' + str(time.time())
# data
dataset = 'stock'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 32
# model
n_layer = 6
n_head = 4
n_embd = 2048
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
meta_vocab_size = 1024

# adamw optimizer
learning_rate = 1e-3 # max learning rate
max_iters = 300 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.99
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 100 # how many steps to warm up for
lr_decay_iters = 2000 # should be ~= max_iters per Chinchilla
min_lr = 1e-4 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

# system
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

master_process = True
seed_offset = 0


if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [8]:
np.random.seed(42)

# 生成50x32大小的数组，值范围在1到10之间
data = np.random.rand(365, n_embd) + 1
# 将NumPy数组转换为pandas DataFrame
train_data = pd.DataFrame(data)
train_data = torch.tensor(train_data.values, dtype=torch.float16)
val_data = train_data
print(train_data)
torch.manual_seed(33)

def get_batch(split):
    data = train_data if split == 'train' else val_data

    indices = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in indices])
    y_indices = [torch.arange(0, len(x[i][0]) // 2) for i, j in enumerate(indices)]
    
    y_list = []
    for i, j in enumerate(indices):
        next_row = data[j + 1:j + 1 + block_size]
        price_chg_cols = [2 * col + 1 for col in y_indices[i]]
        
        topk = 20
        topk_values, topk_indices = next_row[:, price_chg_cols].topk(topk, dim=1)

        random_indices = torch.randint(0, topk, (topk_indices.size(0),))
        selected_indices = topk_indices[torch.arange(topk_indices.size(0)), random_indices]

        y_list.append(selected_indices.clone().detach())
        

    y = torch.stack(y_list)

    x, y = x.to(device), y.to(device)
    return x, y

print(f'batch is {get_batch('train')}')

tensor([[1.3750, 1.9512, 1.7324,  ..., 1.7227, 1.0674, 1.7080],
        [1.5439, 1.0820, 1.4580,  ..., 1.9287, 1.3320, 1.4658],
        [1.0137, 1.0820, 1.2588,  ..., 1.3613, 1.1084, 1.8760],
        ...,
        [1.1025, 1.9570, 1.1562,  ..., 1.0205, 1.3867, 1.1455],
        [1.8369, 1.5430, 1.1621,  ..., 1.6475, 1.8945, 1.8662],
        [1.3232, 1.4258, 1.4482,  ..., 1.1123, 1.4482, 1.7236]],
       dtype=torch.float16)
batch is (tensor([[[1.9658, 1.5693, 1.4180,  ..., 1.8115, 1.6172, 1.8281],
         [1.4512, 1.1787, 1.9180,  ..., 1.1953, 1.9824, 1.8711],
         [1.3467, 1.8301, 1.7646,  ..., 1.6143, 1.1836, 1.6270],
         ...,
         [1.5039, 1.5156, 1.0068,  ..., 1.5703, 1.7275, 1.8740],
         [1.1816, 1.3008, 1.9297,  ..., 1.4561, 1.0137, 1.2529],
         [1.6836, 1.5762, 1.7451,  ..., 1.8350, 1.1289, 1.8818]],

        [[1.6543, 1.7607, 1.1777,  ..., 1.4414, 1.6191, 1.8994],
         [1.3301, 1.8496, 1.0938,  ..., 1.2178, 1.8330, 1.0947],
         [1.4043, 1.1992, 1.

In [9]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=meta_vocab_size, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)


model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

checkpoint = None # free up memory



# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

X, Y = get_batch('train')



logits, loss = model(X, Y)
print(f'logits is {logits}, shape is {logits.shape}')
print(f'loss is {loss}, loss.shape is {loss.shape}')
# print(f'logits={logits}')

# Y.shape
# logits.shape


Initializing a new model from scratch
number of parameters: 304.11M
num decayed parameter tensors: 26, with 304,152,576 parameters
num non-decayed parameter tensors: 13, with 26,624 parameters
using fused AdamW: False
logits is tensor([[[ 1.2725e+00,  1.0312e+00,  5.9189e-01,  ...,  1.1907e+00,
           1.2011e-01,  3.8435e-01],
         [ 7.6813e-01,  6.9559e-01,  1.0309e+00,  ...,  1.9059e+00,
          -5.3035e-01, -6.5802e-01],
         [ 2.9543e-01,  7.5959e-01,  4.0179e-01,  ...,  1.7852e+00,
          -5.0921e-01,  8.8802e-02],
         ...,
         [ 1.1047e+00,  1.7412e+00, -6.0531e-01,  ...,  1.7644e+00,
          -4.8889e-01,  9.7142e-01],
         [-1.0298e+00, -4.3449e-01,  9.8726e-01,  ...,  4.2481e-01,
          -4.7614e-01,  4.0978e-01],
         [ 3.5275e-01, -3.2725e-01,  1.5238e+00,  ...,  1.6243e+00,
           4.8180e-01, -1.2518e-01]],

        [[ 9.5166e-02,  7.9497e-01,  1.5387e-01,  ...,  1.5123e+00,
          -7.2758e-01,  1.3022e-01],
         [-1.0282e-01

In [5]:

# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break




step 0: train loss 7.3203, val loss 7.3350
iter 0: loss 7.3841, time 82255.06ms, mfu -100.00%
iter 1: loss 7.3503, time 8232.30ms, mfu -100.00%
iter 2: loss 7.2621, time 8004.26ms, mfu -100.00%
iter 3: loss 7.1068, time 8474.96ms, mfu -100.00%
iter 4: loss 7.0411, time 8769.57ms, mfu -100.00%
iter 5: loss 6.9560, time 9266.05ms, mfu 0.03%
iter 6: loss 6.9347, time 8372.15ms, mfu 0.03%
iter 7: loss 7.0125, time 9175.79ms, mfu 0.03%
iter 8: loss 6.7482, time 9168.79ms, mfu 0.03%
iter 9: loss 6.7615, time 6976.81ms, mfu 0.03%
iter 10: loss 6.6667, time 6236.87ms, mfu 0.04%
iter 11: loss 6.5987, time 6640.62ms, mfu 0.04%
iter 12: loss 6.5105, time 9155.96ms, mfu 0.04%
iter 13: loss 6.3522, time 9240.67ms, mfu 0.04%
iter 14: loss 6.3059, time 8658.09ms, mfu 0.04%
iter 15: loss 6.4829, time 7034.91ms, mfu 0.04%
iter 16: loss 6.3115, time 6751.88ms, mfu 0.04%
iter 17: loss 6.2224, time 6993.81ms, mfu 0.04%
iter 18: loss 6.1203, time 8319.99ms, mfu 0.04%
iter 19: loss 5.9731, time 8586.31ms, m

KeyboardInterrupt: 